In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence  import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Embedding

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.json to train.json


In [ ]:
import json
import pandas as pd

# Read the file as a string
with open('train.json', 'r') as file:
    data = file.read()

# Replace any occurrences of invalid True/False to lowercase true/false
data = data.replace('True', 'true').replace('False', 'false')

# Now load the JSON data
raw_data = pd.read_json(data)
print(raw_data.head())


<ipython-input-119-1c8d19dba8f4>:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  raw_data = pd.read_json(data)


In [170]:
df = pd.DataFrame(raw_data)

In [171]:
df.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


## **Data Cleaning**

In [172]:
df.isnull().sum()

,0
id,0
cuisine,0
ingredients,0


In [173]:
# Check for duplicates in specific columns, e.g., 'column1', 'column2'
duplicate_count = df['id'].duplicated().sum()
print(duplicate_count)

0


In [174]:
df['ingredients'].sample(10)

,ingredients
13272,"[barbecue sauce, chipotles in adobo, tomato pa..."
38542,"[sesame oil, chinese chives, water, salt, whit..."
16783,"[miso paste, firm tofu, sweet chili sauce, ram..."
7194,"[tomato paste, plain yogurt, peeled fresh ging..."
1339,"[granulated sugar, rice vinegar, gluten free s..."
5358,"[lemon, tomato paste, salmon steaks, reduced s..."
3819,"[vegetable oil, salt, shuck corn, eggs, all-pu..."
13803,"[pepper, purple onion, flavored oil, garlic, r..."
8823,"[cooked rice, garlic powder, paprika, ground t..."
29301,"[sesame seeds, lemon, salmon fillets, mirin, o..."


In [175]:
df['cuisine'].sample(10)

,cuisine
15958,spanish
29589,french
24329,thai
26266,cajun_creole
7613,indian
15922,mexican
20756,indian
23820,filipino
35503,chinese
28559,mexican


In [176]:
df['cuisine'].unique()

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], dtype=object)

In [177]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
wnl = WordNetLemmatizer()

corpus = []
def transform_data(text):
  text = text.lower()
  text = re.sub('[^a-zA-Z]',' ',text)
  text = nltk.word_tokenize(text)
  y = []
  for i in text:
    if i not in set(stopwords.words('english')) and i not in string.punctuation:
      y.append(i)
  text = y[:]
  y.clear()
  for i in text:
    y.append(wnl.lemmatize(i,pos='v'))
  result = ' '.join(y)
  corpus.append(result)
  return result


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [178]:
df['ingredients_transformed'] = df['ingredients'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [179]:
df['ingredients_transformed'] = df['ingredients_transformed'].apply(transform_data)

In [180]:
corpus[0]

'romaine lettuce black olives grape tomatoes garlic pepper purple onion season garbanzo bean feta cheese crumble'

In [181]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
le = LabelEncoder()
y = to_categorical(le.fit_transform(df['cuisine']))

## **One Hot Representation**

In [182]:
voc_size = 5000
onehot_rep = [one_hot(words,voc_size) for words in corpus]

In [191]:
len(onehot_rep)

39774

## **Embedding layer**

In [183]:
sent_length = 30
embedded_doc =  pad_sequences(onehot_rep,padding ='pre',maxlen = sent_length)

In [184]:
embedded_doc.shape

(39774, 30)

In [192]:
X = np.array(embedded_doc)

In [193]:
X.shape

(39774, 30)

In [186]:
y.shape

(39774, 20)

In [194]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)


## Model Building

In [195]:

model = Sequential([
    layers.Embedding(voc_size, dim),
    layers.Bidirectional(layers.LSTM(100, return_sequences=True)),
    layers.Dropout(0.3),
    layers.Bidirectional(layers.LSTM(50)),
    layers.Dropout(0.3),
    layers.Dense(100, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')
])


In [196]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [197]:
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 98s 46ms/step - accuracy: 0.4624 - loss: 1.7624 - val_accuracy: 0.6827 - val_loss: 1.0449
Epoch 2/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 139s 45ms/step - accuracy: 0.6917 - loss: 0.9932 - val_accuracy: 0.7140 - val_loss: 0.9358
Epoch 3/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 141s 44ms/step - accuracy: 0.7348 - loss: 0.8685 - val_accuracy: 0.7208 - val_loss: 0.9226
Epoch 4/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 143s 45ms/step - accuracy: 0.7643 - loss: 0.7662 - val_accuracy: 0.7453 - val_loss: 0.8524
Epoch 5/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 88s 44ms/step - accuracy: 0.7802 - loss: 0.7202 - val_accuracy: 0.7480 - val_loss: 0.8447
Epoch 6/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 143s 45ms/step - accuracy: 0.7938 - loss: 0.6742 - val_accuracy: 0.7549 - val_loss: 0.8152
Epoch 7/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 141s 44ms/step - accuracy: 0.8101 - loss: 0.6242 - val_accuracy: 0.7539 - val_loss: 0.8319
Epoch 8/10
1989/1989 ━━━━━━━━━━━━━━━━━━━━ 143s 45ms/step - accuracy: 0.

In [198]:
from sklearn.metrics import accuracy_score
y_pred =model.predict(X_test)
y_pred = np.where(y_pred>0.5,1,0)  ## AUC Roc Curveaccuracy =accuracy_score(y_test,y_pred)
print(accuracy_score(y_test,y_pred))

249/249 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
0.7175361407919547
